## Setup
All the imports. simutils contains a set of functions I created for generating synthetic data for these experiments.

In [1]:
#!pip install statsmodels
#!pip install scikit-learn
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install scipy

import simutils as sim
import numpy as np
import pandas as pd

## Predictor Variables of Non-Interest
First, create some synthetic age, credit score and arbitrary hypothetically relevant personality characteristic values. Most of these variables are nuisance or confounding variables, but will partially determine both the predictor of interest and the dependent variable.

Age will be normally distributed (M=35, SD=10) and range from 18 to 78. Credit scores are 700-730 on average, but tend to be lower for younger people, and are reduced by up to 40 points for the youngest, relative to the oldest. The hypothetical personality variable will reflect whatever set of intrinsic characteristics (e.g., inquisitiveness, intelligence) that are normally distributed and might drive the behaviour of interest.

In [2]:
N=50000
ages=sim.generate_ages(35, 10, N)
credit_scores=sim.generate_credit_scores(ages)
personality=np.random.normal(loc=0, scale=1, size=N)
#create data frame out of factors
Z=pd.DataFrame({
    'age': ages,
    'credit_score': credit_scores,
    'personality': personality
    })
Z.head()

,age,credit_score,personality
0,38,721,-0.444072
1,24,631,-0.116099
2,42,751,-1.038549
3,44,763,-1.086927
4,18,572,1.264644


## Predictor of Interest
The independent measure will be the number of recorded interactions with product information pages, obtained from cookies, since monitoring was implemented.

Our goal will be to determine the causal relationship between clicking behavior and the dependent variable, personal wealth. This relationship can be quantified by computing the coefficient or weight in a regression or machine learning model that predicts wealth from clicks.

Clicking behavior will be influenced by all three of the variables in Z. These influences will be non-linear, which would make a linear regression-based approach a poor choice, though the number of clicks will be a linear combination of all three influences. I will explicitly determine each confound's influence on an individual's clicking behavior, making it easier to establish the ground-truth.

In [3]:
from scipy.stats import randint

clicks=sim.generate_clicks(Z)
clicks.columns=['age_delta', 'csv_delta', 'personality_delta']
#external random factors are responsible for the baseline number of interactions
clicks['baseline'] = randint.rvs(10,50, size=N)
clicks['total'] = clicks.apply(np.sum, axis=1)
clicks.head()

,age_delta,csv_delta,personality_delta,baseline,total
0,44,8,5,22,79
1,33,32,8,49,122
2,60,11,0,47,118
3,62,15,0,26,103
4,-1,46,22,43,110


From the perspective of an all-knowing oracle, we can isolate the total number of clicks for each person that were not attributable to either age or credit score value:

In [4]:
clean_clicks=pd.DataFrame(clicks['personality_delta']+clicks['baseline'])
clean_clicks.columns=['i_total']

## The Dependent Variable: Wealth/Value
Now to create the dependent variable, portfolio value, we can use a similar procedure. The DV value will be caused in part by the number of clicks, as well as age and credit score and we can see whether we can tease apart these components. **Every click causes an increases a person's wealth by $500**, and this is the causal parameter we are trying to discover.

In [5]:
#baseline wealth increases as individuals get older until ~75, at which point it drops off
age_wealth=Z.apply(lambda row: sim.age_worth(row[0]), axis=1)
#baseline wealth increases for people with better credit scores, who can get better rates
csv_wealth=Z.apply(lambda row: sim.csv_worth(row[1]), axis=1)
#wealth increases caused by the behavior we care about (clicks), itself influenced by age and credit
click_wealth=clicks.apply(lambda row: sim.clicks_worth(row[4]), axis=1)
#wealth attributable to all the other random chance factors
circumstance_wealth=clicks.apply(lambda row: sim.circumstance_worth(), axis=1)
wealth=pd.concat([age_wealth, csv_wealth, click_wealth, circumstance_wealth], axis=1)
wealth.columns=['age_wealth', 'csv_wealth', 'click_wealth', 'circumstance_wealth']
wealth["total_wealth"]=wealth.apply(np.sum, axis=1)
wealth.head()


,age_wealth,csv_wealth,click_wealth,circumstance_wealth,total_wealth
0,18379,54368,39495,50511,162753
1,15229,39603,60993,46389,162214
2,35530,60454,58996,49019,203999
3,38973,63080,51499,35098,188650
4,11037,32217,54991,35384,133629


Like for clicks, I want the oracle measure of the wealth attributable to those clicks that were not driven by age and credit score:

In [6]:
clean_wealth=clean_clicks.apply(lambda row: sim.clicks_worth(row[0]), axis=1)

## Naive Multiple Regression
So the goal is to use the customer's interaction data ("clicks") to explain the value of their portfolio ("total wealth"). In this scenario, we have reason to believe that age and credit score affect both the number of interactions and the customer's total wealth and need to remove these confounding effects. Before we do so, let's run a naive multiple regression, paying attention to the coefficient for the clicks variable, to see the impact of the confounding variables on our parameter estimate.

In [7]:
import statsmodels.formula.api as smf
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict

y=wealth["total_wealth"]
x=clicks["total"]
basedf=pd.DataFrame({'y': y, 'clicks': x})
basedf["age"]=Z["age"]
basedf["csv"]=Z["credit_score"]

naive_model = smf.ols(formula='y ~ 1 + clicks + age + csv', data = basedf).fit()
print(naive_model.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.906
Method:                 Least Squares   F-statistic:                 1.606e+05
Date:                Mon, 06 Nov 2023   Prob (F-statistic):               0.00
Time:                        14:57:19   Log-Likelihood:            -5.3157e+05
No. Observations:               50000   AIC:                         1.063e+06
Df Residuals:                   49996   BIC:                         1.063e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.022e+05   2785.385     36.708      0.0

In the above model summary, the coefficient estimate for clicks is quite biased: we know the true value to be 500, so the model is greatly overestimating the impact of clicking. There's also an indicator of potential multicollinearity, which is not at all surprising, since age and csv drive the number of clicks.

## Double Machine Learning
Now let's use double machine learning to try to remove the confounding effects of age and credit score. This is done by using the confounders to predict the variables we care about (clicks and wealth), and calculating the residuals. The residualized values are the components of the variables of interest that couldn't be adequately predicted by the confounders, and are orthogonal with respect to the confounding variables.

The particular ML algorithm used isn't critical, as long as it is appropriate for your variables. Here, I am predicting continuous integer values, and had never before had an opportunity to try out a boosted decision tree algorithm. For categorical data, an SVM or even neural network might be appropriate.

In [8]:
M_clicks=GradientBoostingRegressor()
M_wealth=GradientBoostingRegressor()

#use cross-validation values to obtain residuals
residualized_y = y-cross_val_predict(M_wealth, Z[["age", "credit_score"]], y, cv=3)
residualized_x = x-cross_val_predict(M_clicks, Z[["age", "credit_score"]], x, cv=3)
df=pd.DataFrame()
df["Y_hat"]=residualized_y
df["X_hat"]=residualized_x
df["clean_X"]=clean_clicks
df["dirty_Y"]=wealth["click_wealth"]
df["clean_Y"]=clean_wealth

#fit the residualized data and check out the model prediction
DML_model = smf.ols(formula='Y_hat ~ 1 + X_hat', data = df).fit()
print(DML_model.summary())


                            OLS Regression Results                            
Dep. Variable:                  Y_hat   R-squared:                       0.712
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                 1.233e+05
Date:                Mon, 06 Nov 2023   Prob (F-statistic):               0.00
Time:                        14:57:42   Log-Likelihood:            -4.9725e+05
No. Observations:               50000   AIC:                         9.945e+05
Df Residuals:                   49998   BIC:                         9.945e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5582     22.558      0.025      0.9

## Oracle Models
The double-ML model using the orthogonalized values are much closer to the true parameter value.

From the perspective of an all-knowing oracle, we can compare the double-ML results against models using the ground-truth oracle data. In my first attempt, I modeled wealth values that were computed only using the total number of clicks (which itself was influenced by age and credit score), which I then realized doesn't quite capture what double-ML was supposed to have done:

In [9]:
#fit the number of clicks unrelated to age/credit score to the wealth attributable to clicks alone
Oracle_model_1 = smf.ols(formula='dirty_Y ~ 1 + clean_X', data = df).fit()
print(Oracle_model_1.summary())


                            OLS Regression Results                            
Dep. Variable:                dirty_Y   R-squared:                       0.484
Model:                            OLS   Adj. R-squared:                  0.484
Method:                 Least Squares   F-statistic:                 4.684e+04
Date:                Mon, 06 Nov 2023   Prob (F-statistic):               0.00
Time:                        14:57:42   Log-Likelihood:            -5.1883e+05
No. Observations:               50000   AIC:                         1.038e+06
Df Residuals:                   49998   BIC:                         1.038e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.227e+04     96.617    333.962      0.0

I followed up by repeating the model, using the click behavior not attributable to age and credit score to generate the wealth attributable to the clicking behavior that is also not attributable to age and credit score:

In [10]:
#fit the number of clicks unrelated to age/credit score to the wealth attributable to clicks alone
Oracle_model_2 = smf.ols(formula='clean_Y ~ 1 + clean_X', data = df).fit()
print(Oracle_model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                clean_Y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.542e+10
Date:                Mon, 06 Nov 2023   Prob (F-statistic):               0.00
Time:                        14:57:42   Log-Likelihood:            -1.5850e+05
No. Observations:               50000   AIC:                         3.170e+05
Df Residuals:                   49998   BIC:                         3.170e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.5121      0.072     -7.145      0.0

## Comparing Orthogonalized Scores to Oracle Values
A final question to consider is whether the orthogonalized scores are equivalent to "clean" data; that is, whether they map to the ground-truth oracle data without the influence of the nuisance variables. Consider that we have access to the components of each person's click behaviour that are independent of age and credit score. We can quantify the correspondence of these values with the residualized scores. 

In [12]:
from scipy import stats
import matplotlib.pyplot as plt

res=stats.spearmanr(residualized_x, clean_clicks)
bad=stats.spearmanr(residualized_x, clicks["total"])
print(f"The residualized values are correlated at {int(res.statistic*100)}% with the true values and {int(bad.statistic *100)}% with the dirty values")

The residualized values are correlated at 95% with the true values and 71% with the dirty values
